In [2]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)
from datetime import datetime, date, time, timezone
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

import mglearn
from IPython.display import display

# Dataset 3: FedEx orders with box dim

In [18]:
cols_list = ['Tracking number','Reference 2','Dim length', 'Dim width', 'Dim height','Scale weight', 'Rated weight']
box_size_1= pd.read_csv('new_data/2019_07_01-2019_08_31-Shipment_details.csv',usecols = cols_list)
box_size_2= pd.read_csv('new_data/2019_09_01-2019_12_31-Shipment_details.csv',usecols = cols_list)
box_size_3= pd.read_csv('new_data/2020_01_01-2020_04_30-Shipment_details.csv',usecols = cols_list)
box_size_4= pd.read_csv('new_data/2020_05_01-2020_08_31-Shipment_details.csv',usecols = cols_list)
box_size_5= pd.read_csv('new_data/2019_01_01-2019_04_30-Shipment_details.csv',usecols = cols_list)
box_size_6= pd.read_csv('new_data/2019_05_01-2019_06_30-Shipment_details.csv',usecols = cols_list)


box_size = pd.concat([box_size_1,box_size_2,box_size_3,box_size_4,box_size_5,box_size_6], ignore_index = True).copy()


In [26]:
box_size.shape

(578853, 7)

In [20]:
box_size.head(5)

,Tracking number,Scale weight,Rated weight,Dim length,Dim width,Dim height,Reference 2
0,788197948826,1.0,2.0,10.0,7.0,4.0,QG9N-PSMM
1,788192874690,4.0,4.0,14.0,10.0,5.0,VN9W-1SLQ
2,788228260658,1.0,1.0,8.0,5.0,3.0,ZL1M-SFX9
3,788198403764,4.0,5.0,15.0,11.0,5.0,9LSM-L1FN
4,788229760154,1.0,2.0,10.0,7.0,4.0,F3S9-L18F


In [24]:
box_size.isna().sum()

Tracking number        0
Scale weight          34
Rated weight           0
Dim length             0
Dim width              0
Dim height             0
Reference 2        26082
dtype: int64

In [25]:
box_size.dropna(subset=['Dim height','Reference 2'], inplace = True)

# Dataset 2: Order Line Item

In [54]:
line1 = pd.read_csv('new_data/Jan2019_Jun2019_order_line.csv')
line2 = pd.read_csv('new_data/Jul2019_Aug2020_order_line.csv')
line = pd.concat([line1,line2], ignore_index = True).copy()

In [55]:
line.shape

(6934053, 3)

In [66]:
line.head(3)

,order_number,variant_id,quantity
0,PV7B-Q92J,50404,1
1,PV7B-Q92J,57308,1
2,PV7B-Q92J,62133,1


In [49]:
sample_list = pd.read_csv('new_data/sample_list.csv')
sample_list['indicator']=1
sample_list.head(4)

,variant_id,indicator
0,47348,1
1,47349,1
2,47640,1
3,47869,1


In [60]:
line_no_sample = pd.merge(line, sample_list, how = 'left', on ='variant_id')
line_no_sample.head(4)

,order_number,variant_id,quantity,indicator
0,PV7B-Q92J,50404,1,NaN
1,PV7B-Q92J,57308,1,NaN
2,PV7B-Q92J,62133,1,NaN
3,PV7B-Q92J,53174,1,NaN


In [64]:
line_no_sample = line_no_sample.loc[line_no_sample['indicator'] !=1].copy()

line_no_sample.drop(columns=['indicator'], inplace = True)

line_no_sample.shape

In [89]:
# Finding list of total SKUs sold, with and without dimenstion

item=line_no_sample[['variant_id']].drop_duplicates()

item.shape

In [91]:
merge_dim = pd.merge(item,item_dim, how='left', on ='variant_id')

In [103]:
merge_dim = merge_dim.sort_values(['variant_id']).copy()

In [104]:
merge_dim.head(3)

,variant_id,dim_packaging_length_mm,dim_packaging_width_mm,dim_packaging_height_mm
2493,126,18.0,18.0,172.0
6167,127,18.0,18.0,172.0
6495,128,18.0,18.0,172.0


In [134]:
item_names = pd.read_csv('new_data/all_item_names.csv')
print(item_names.shape)
item_names.head(4)

(15283, 7)


,variant_id,product_name,variant_name,name_string,brand_name,brand_id,weight
0,126,Perfect Brow Pencil,Taupe,anastasia-beverly-hills-perfect-brow-pencil-taupe,Anastasia Beverly Hills,117,0.03
1,127,Perfect Brow Pencil,Blonde,anastasia-beverly-hills-perfect-brow-pencil-bl...,Anastasia Beverly Hills,117,0.03
2,128,Perfect Brow Pencil,Caramel,anastasia-beverly-hills-perfect-brow-pencil-ca...,Anastasia Beverly Hills,117,0.03
3,129,Perfect Brow Pencil,Medium Brown,anastasia-beverly-hills-perfect-brow-pencil-me...,Anastasia Beverly Hills,117,0.03


In [135]:
all_item = pd.merge(merge_dim,item_names, how = 'left', on = 'variant_id')
all_item

,variant_id,dim_packaging_length_mm,dim_packaging_width_mm,dim_packaging_height_mm,product_name,variant_name,name_string,brand_name,brand_id,weight
0,126,18.0,18.0,172.0,Perfect Brow Pencil,Taupe,anastasia-beverly-hills-perfect-brow-pencil-taupe,Anastasia Beverly Hills,117.0,0.03
1,127,18.0,18.0,172.0,Perfect Brow Pencil,Blonde,anastasia-beverly-hills-perfect-brow-pencil-bl...,Anastasia Beverly Hills,117.0,0.03
2,128,18.0,18.0,172.0,Perfect Brow Pencil,Caramel,anastasia-beverly-hills-perfect-brow-pencil-ca...,Anastasia Beverly Hills,117.0,0.03
3,129,18.0,18.0,172.0,Perfect Brow Pencil,Medium Brown,anastasia-beverly-hills-perfect-brow-pencil-me...,Anastasia Beverly Hills,117.0,0.03
4,130,18.0,18.0,172.0,Perfect Brow Pencil,Dark Brown,anastasia-beverly-hills-perfect-brow-pencil-da...,Anastasia Beverly Hills,117.0,0.03
...,...,...,...,...,...,...,...,...,...,...
14907,68804,NaN,NaN,NaN,Magic Star Luminous Setting Powder,Topaz,jeffree-star-cosmetics-magic-star-luminous-set...,Jeffree Star Cosmetics,2944.0,0.17
14908,68805,NaN,NaN,NaN,Magic Star Luminous Setting Powder,Honey,jeffree-star-cosmetics-magic-star-luminous-set...,Jeffree Star Cosmetics,2944.0,0.17
14909,68806,NaN,NaN,NaN,Magic Star Luminous Setting Powder,Suede,jeffree-star-cosmetics-magic-star-luminous-set...,Jeffree Star Cosmetics,2944.0,0.17
14910,68807,NaN,NaN,NaN,Magic Star Luminous Setting Powder,Caramel,jeffree-star-cosmetics-magic-star-luminous-set...,Jeffree Star Cosmetics,2944.0,0.17


In [136]:
brand = 'Jeffree Star Cosmetics'

In [137]:
conditions = all_item['product_name'].str.contains('Setting Powder', na=False) &  (all_item['brand_name'] == brand)

In [138]:
all_item.loc[conditions]

,variant_id,dim_packaging_length_mm,dim_packaging_width_mm,dim_packaging_height_mm,product_name,variant_name,name_string,brand_name,brand_id,weight
11428,64201,76.0,76.0,33.0,Magic Star Setting Powder,Banana,jeffree-star-cosmetics-magic-star-setting-powd...,Jeffree Star Cosmetics,2944.0,0.17
11429,64202,76.0,76.0,33.0,Magic Star Setting Powder,Beige,jeffree-star-cosmetics-magic-star-setting-powd...,Jeffree Star Cosmetics,2944.0,0.17
11430,64203,76.0,76.0,33.0,Magic Star Setting Powder,Caramel,jeffree-star-cosmetics-magic-star-setting-powd...,Jeffree Star Cosmetics,2944.0,0.17
11431,64204,76.0,76.0,33.0,Magic Star Setting Powder,Fair,jeffree-star-cosmetics-magic-star-setting-powd...,Jeffree Star Cosmetics,2944.0,0.17
11432,64205,76.0,76.0,33.0,Magic Star Setting Powder,Rose,jeffree-star-cosmetics-magic-star-setting-powd...,Jeffree Star Cosmetics,2944.0,0.17
11434,64207,76.0,76.0,33.0,Magic Star Setting Powder,Suede,jeffree-star-cosmetics-magic-star-setting-powd...,Jeffree Star Cosmetics,2944.0,0.17
11435,64208,76.0,76.0,33.0,Magic Star Setting Powder,Topaz,jeffree-star-cosmetics-magic-star-setting-powd...,Jeffree Star Cosmetics,2944.0,0.17
11440,64213,76.0,76.0,33.0,Magic Star Setting Powder,Translucent,jeffree-star-cosmetics-magic-star-setting-powd...,Jeffree Star Cosmetics,2944.0,0.17
14902,68799,NaN,NaN,NaN,Magic Star Luminous Setting Powder,Translucent,jeffree-star-cosmetics-magic-star-luminous-set...,Jeffree Star Cosmetics,2944.0,0.17
14903,68800,NaN,NaN,NaN,Magic Star Luminous Setting Powder,Natural,jeffree-star-cosmetics-magic-star-luminous-set...,Jeffree Star Cosmetics,2944.0,0.17


# Dataset 1: Item 3D

In [40]:
cols = ['variant_id','dim_packaging_length_mm','dim_packaging_width_mm','dim_packaging_height_mm']
item_dim = pd.read_csv('new_data/variant_specs.csv', usecols = cols)

In [41]:
item_dim.shape

(11920, 4)

In [42]:
item_dim.head(3)

,variant_id,dim_packaging_length_mm,dim_packaging_width_mm,dim_packaging_height_mm
0,126,18.0,18.0,172.0
1,127,18.0,18.0,172.0
2,128,18.0,18.0,172.0


In [45]:
item_dim.isna().sum()

variant_id                 0
dim_packaging_length_mm    0
dim_packaging_width_mm     0
dim_packaging_height_mm    0
dtype: int64

In [44]:
item_dim.dropna(subset=['dim_packaging_length_mm'], inplace = True)